In [ ]:
def create_and_save_df_of_reccomended_lists(submit_df):

    users_for_predictions = submit_df.user_id.unique()
    cart_user = submit_df[['user_id', 'cart_id']]

    list_of_carts = pd.DataFrame(columns=['user_id','carts'])

    for userid in users_for_predictions:
        
        carts = cart_user[cart_user["user_id"]==userid].cart_id.to_list()

        list_of_carts=pd.concat([pd.DataFrame([
                {   
                    'user_id': userid, 
                    'carts': carts
                }]),
                list_of_carts],ignore_index=True)
        
    lists_rec = list_of_carts.sort_values(by='user_id').reset_index(drop=True)
    lists_rec.to_csv('lists_rec.csv', encoding='utf-8')
    return lists_rec


lists_rec = create_and_save_df_of_reccomended_lists(submit_df)
lists_rec
# lists_rec = pd.read_csv('lists_rec2.csv', encoding='utf-8')
# lists_rec.drop(columns=['Unnamed: 0'], inplace=True)
# lists_rec.set_index('user_id', inplace=True)

In [ ]:
# Все юзеры во всем тестовом датасете
unique_users_all_ds = raw_data.user_id.unique()
unique_users_all_ds.shape[0]

# Уникальные группы во всем тестовом датасете
unique_carts_all_ds = raw_data.cart.unique()
unique_carts_all_ds.shape[0]

In [ ]:
carts_ids = unique_carts_all_ds

score_1_level = 0.6
score_0_level = 0.1
score_err_level = 0.01

k = unique_carts_all_ds.shape[0]

df_test_results_1 = pd.DataFrame(columns=['user_id','cart_id','score'])

def concat_to_df(df, userid, cartid, score):
    df=pd.concat([pd.DataFrame([
                    {   
                        'user_id': userid, 
                        'cart_id': cartid, 
                        'score': score
                    }]),
                    df],ignore_index=True)
    return df

for userid in users_ids:
    # print(userid)

    ids, scores = calculate_recommendations_for_user(userid, model, k)
    ids = ids.tolist()
    recs_for_user = pd.DataFrame({'score': scores}, index=ids)
    
    print(ids)
    # print(recs_for_user)

    for cartid in ids:
        if cartid != 0:
            # print(cartid)
            score = recs_for_user.loc[cartid,'score']
            # print(score)

            df_test_results_1=concat_to_df(df_test_results_1, userid, cartid, score)


df_test_results_1.sort_values(by='score', ascending=False).head(15)  